In [2]:
!pip install pycocotools


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 13.7 MB/s eta 0:00:00


In [1]:
import os
import json
import numpy as np
import torch
from transformers import AutoProcessor, BlipModel
from PIL import Image

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# File paths and directories for VCR dataset
IMAGE_DIR = '/kaggle/input/vcr-dset/vcr/vcr1images/vcr1images'
ANNOTATION_FILE = '/kaggle/input/vcr-dset/vcr/vcr1annots/train.jsonl'
OUTPUT_PATH = '/kaggle/working/'

# Load VCR annotations
annotations = []
with open(ANNOTATION_FILE, 'r') as f:
    for line in f:
        annotations.append(json.loads(line))

# Load the pretrained BLIP Processor and BLIP model with your Hugging Face API token


model = BlipModel.from_pretrained("Salesforce/blip-image-captioning-base")

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

# Move model to GPU if available
model.to(device)

# Function to extract features using BLIP model
def extract_features(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt")
        
        # Move inputs to GPU if available
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.get_image_features(**inputs)
            features = outputs.squeeze().cpu().numpy()
        
        return features
    
    except Exception as e:
        print(f"Error extracting features for image {image_path}: {str(e)}")
        return np.array([])  # Return empty array on error

# Feature extraction loop
features_list = []
labels_list = []

# Check if there are any previously saved features and labels
if os.path.exists(os.path.join(OUTPUT_PATH, "checkpoint.npy")):
    checkpoint = np.load(os.path.join(OUTPUT_PATH, "checkpoint.npy"), allow_pickle=True).item()
    start_index = checkpoint['last_processed_index']
    features_list = checkpoint['features_list']
    labels_list = checkpoint['labels_list']
    print(f"Checkpoint found. Resuming from index {start_index}.")
else:
    start_index = 0

for i, ann in enumerate(annotations[start_index:], start=start_index):
    image_filename = ann['img_fn']
    image_path = os.path.join(IMAGE_DIR, image_filename)
    try:
        features = extract_features(image_path)
        if features.size == 0:
            continue
        labels = ann['answer_label']  # Adjust this part based on actual label structure in VCR
        features_list.append(features)
        labels_list.append(labels)
    except FileNotFoundError:
        print(f"File not found: {image_path}, skipping.")
    except Exception as e:
        print(f"An error occurred with file {image_path}: {e}")
    
    if (i + 1) % 1000 == 0:
        print(f"{i + 1} images processed")

    # Save checkpoint every 500 images processed
    if (i + 1) % 500 == 0:
        checkpoint = {
            'last_processed_index': i + 1,
            'features_list': features_list,
            'labels_list': labels_list
        }
        np.save(os.path.join(OUTPUT_PATH, "checkpoint.npy"), checkpoint)

# Determine all unique labels (categories)
unique_labels = sorted(set(labels_list))

# Convert labels to one-hot encoding
label_map = {label: idx for idx, label in enumerate(unique_labels)}

def convert_to_one_hot(label, label_map):
    one_hot_labels = np.zeros(len(label_map))
    one_hot_labels[label_map[label]] = 1
    return one_hot_labels

# Convert labels_list to one-hot encoding
one_hot_labels_list = [convert_to_one_hot(label, label_map) for label in labels_list]

# Reshape extracted features and labels
train_features_array = np.array(features_list)
train_labels_array = np.array(one_hot_labels_list)

# Save the features and labels
np.save(os.path.join(OUTPUT_PATH, "blip_train_features.npy"), train_features_array)
np.save(os.path.join(OUTPUT_PATH, "blip_train_labels.npy"), train_labels_array)

print("Features array shape:", train_features_array.shape)
print("Labels array shape:", train_labels_array.shape)

# Remove checkpoint file if processing completed successfully
if os.path.exists(os.path.join(OUTPUT_PATH, "checkpoint.npy")):
    os.remove(os.path.join(OUTPUT_PATH, "checkpoint.npy"))
    print("Removed checkpoint file.")


2024-06-01 06:38:55.276005: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-01 06:38:55.276104: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-01 06:38:55.414938: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BlipModel were not initialized from the model checkpoint at Salesforce/blip-image-captioning-base and are newly initialized: ['logit_scale', 'text_model.embeddings.LayerNorm.bias', 'text_model.embeddings.LayerNorm.weight', 'text_model.embeddings.position_embeddings.weight', 'text_model.embeddings.word_embeddings.weight', 'text_model.encoder.layer.0.attention.output.LayerNorm.bias', 'text_model.encoder.layer.0.attention.output.LayerNorm.weight', 'text_model.encoder.layer.0.attention.output.dense.bias', 'text_model.encoder.layer.0.attention.output.dense.weight', 'text_model.encoder.layer

...

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1000 images processed
2000 images processed
3000 images processed
4000 images processed
5000 images processed
6000 images processed
7000 images processed
8000 images processed
9000 images processed
10000 images processed
11000 images processed
12000 images processed
13000 images processed
Error extracting features for image /kaggle/input/vcr-dset/vcr/vcr1images/vcr1images/movieclips_Roman_J._Israel_Esq./c9oE47YW6YM@3.jpg: [Errno 2] No such file or directory: '/kaggle/input/vcr-dset/vcr/vcr1images/vcr1images/movieclips_Roman_J._Israel_Esq./c9oE47YW6YM@3.jpg'
Error extracting features for image /kaggle/input/vcr-dset/vcr/vcr1images/vcr1images/movieclips_Roman_J._Israel_Esq./c9oE47YW6YM@3.jpg: [Errno 2] No such file or directory: '/kaggle/input/vcr-dset/vcr/vcr1images/vcr1images/movieclips_Roman_J._Israel_Esq./c9oE47YW6YM@3.jpg'
Error extracting features for image /kaggle/input/vcr-dset/vcr/vcr1images/vcr1images/movieclips_The_Stoning_of_Soraya_M./5CuKjYdDd50@7.jpg: [Errno 2] No such file

In [1]:
import os
import json
import numpy as np
import torch
from transformers import AutoProcessor, BlipModel
from PIL import Image

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# File paths and directories for VCR dataset
IMAGE_DIR = '/kaggle/input/vcr-dset/vcr/vcr1images/vcr1images'
ANNOTATION_FILE = '/kaggle/input/vcr-dset/vcr/vcr1annots/val.jsonl'
OUTPUT_PATH = '/kaggle/working/'

# Load VCR annotations
annotations = []
with open(ANNOTATION_FILE, 'r') as f:
    for line in f:
        annotations.append(json.loads(line))

# Load the pretrained BLIP Processor and BLIP model with your Hugging Face API token


model = BlipModel.from_pretrained("Salesforce/blip-image-captioning-base")

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

# Move model to GPU if available
model.to(device)

# Function to extract features using BLIP model
def extract_features(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt")
        
        # Move inputs to GPU if available
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.get_image_features(**inputs)
            features = outputs.squeeze().cpu().numpy()
        
        return features
    
    except Exception as e:
        print(f"Error extracting features for image {image_path}: {str(e)}")
        return np.array([])  # Return empty array on error

# Feature extraction loop
features_list = []
labels_list = []

# Check if there are any previously saved features and labels
if os.path.exists(os.path.join(OUTPUT_PATH, "checkpoint.npy")):
    checkpoint = np.load(os.path.join(OUTPUT_PATH, "checkpoint.npy"), allow_pickle=True).item()
    start_index = checkpoint['last_processed_index']
    features_list = checkpoint['features_list']
    labels_list = checkpoint['labels_list']
    print(f"Checkpoint found. Resuming from index {start_index}.")
else:
    start_index = 0

for i, ann in enumerate(annotations[start_index:], start=start_index):
    image_filename = ann['img_fn']
    image_path = os.path.join(IMAGE_DIR, image_filename)
    try:
        features = extract_features(image_path)
        if features.size == 0:
            continue
        labels = ann['answer_label']  # Adjust this part based on actual label structure in VCR
        features_list.append(features)
        labels_list.append(labels)
    except FileNotFoundError:
        print(f"File not found: {image_path}, skipping.")
    except Exception as e:
        print(f"An error occurred with file {image_path}: {e}")
    
    if (i + 1) % 1000 == 0:
        print(f"{i + 1} images processed")

    # Save checkpoint every 500 images processed
    if (i + 1) % 500 == 0:
        checkpoint = {
            'last_processed_index': i + 1,
            'features_list': features_list,
            'labels_list': labels_list
        }
        np.save(os.path.join(OUTPUT_PATH, "checkpoint.npy"), checkpoint)

# Determine all unique labels (categories)
unique_labels = sorted(set(labels_list))

# Convert labels to one-hot encoding
label_map = {label: idx for idx, label in enumerate(unique_labels)}

def convert_to_one_hot(label, label_map):
    one_hot_labels = np.zeros(len(label_map))
    one_hot_labels[label_map[label]] = 1
    return one_hot_labels

# Convert labels_list to one-hot encoding
one_hot_labels_list = [convert_to_one_hot(label, label_map) for label in labels_list]

# Reshape extracted features and labels
val_features_array = np.array(features_list)
val_labels_array = np.array(one_hot_labels_list)

# Save the features and labels
np.save(os.path.join(OUTPUT_PATH, "blip_val_features.npy"), val_features_array)
np.save(os.path.join(OUTPUT_PATH, "blip_val_labels.npy"), val_labels_array)

print("Features array shape:", val_features_array.shape)
print("Labels array shape:", val_labels_array.shape)

# Remove checkpoint file if processing completed successfully
if os.path.exists(os.path.join(OUTPUT_PATH, "checkpoint.npy")):
    os.remove(os.path.join(OUTPUT_PATH, "checkpoint.npy"))
    print("Removed checkpoint file.")


2024-06-01 10:42:29.589701: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-01 10:42:29.589876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-01 10:42:29.715376: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BlipModel were not initialized from the model checkpoint at Salesforce/blip-image-captioning-base and are newly initialized: ['logit_scale', 'text_model.embeddings.LayerNorm.bias', 'text_model.embeddings.LayerNorm.weight', 'text_model.embeddings.position_embeddings.weight', 'text_model.embeddings.word_embeddings.weight', 'text_model.encoder.layer.0.attention.output.LayerNorm.bias', 'text_model.encoder.layer.0.attention.output.LayerNorm.weight', 'text_model.encoder.layer.0.attention.output.dense.bias', 'text_model.encoder.layer.0.attention.output.dense.weight', 'text_model.encoder.layer

...

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1000 images processed
2000 images processed
3000 images processed
4000 images processed
5000 images processed
6000 images processed
7000 images processed
8000 images processed
9000 images processed
12000 images processed
Error extracting features for image /kaggle/input/vcr-dset/vcr/vcr1images/vcr1images/movieclips_Waiting.../3dzw4t_dB5M@2.jpg: [Errno 2] No such file or directory: '/kaggle/input/vcr-dset/vcr/vcr1images/vcr1images/movieclips_Waiting.../3dzw4t_dB5M@2.jpg'
Error extracting features for image /kaggle/input/vcr-dset/vcr/vcr1images/vcr1images/movieclips_Waiting.../QVw-cRLFWOI@8.jpg: [Errno 2] No such file or directory: '/kaggle/input/vcr-dset/vcr/vcr1images/vcr1images/movieclips_Waiting.../QVw-cRLFWOI@8.jpg'
Error extracting features for image /kaggle/input/vcr-dset/vcr/vcr1images/vcr1images/movieclips_Waiting.../QVw-cRLFWOI@8.jpg: [Errno 2] No such file or directory: '/kaggle/input/vcr-dset/vcr/vcr1images/vcr1images/movieclips_Waiting.../QVw-cRLFWOI@8.jpg'
Error extracting 